In [2]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils import data

import numpy as np

import os
import sys

import matplotlib.pyplot as plt

sys.path.append(os.path.abspath('..'))
from LoadRealRunningJumping import *
from LoadSyntheticRunningJumping import *

from dataLoader import *

import torch.fft as fft

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:55: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [6]:
real_sample = daghar_load_dataset(class_name='run', path='../DAGHAR_GANs/')
synthetic_sample= Single_Class_Synthetic_Dataset( path = '../pre-trained-models/daghar_50000_30_100/run_50000_D_30_2024_10_18_01_39_30/Model/checkpoint',
                                                      sample_size = len(real_sample),
                                                      seq_len = 30)

real_sample, synthetic_sample = real_sample[:][0], synthetic_sample[:][0]
np.asarray(real_sample).squeeze().shape

return single class data and labels, class is run
data shape is (16260, 3, 1, 30)
label shape is (16260,)
Checkpoint epochs: 50


/workspaces/container-workspace/tts-gan/LoadSyntheticRunningJumping.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckp = torch.load(path,  map_location=torch.device('c

(16260, 3, 1, 30)
(16260, 1)


(16260, 3, 30)

In [11]:
batch_size, _, _ = np.asarray(real_sample).squeeze().shape

In [12]:
batch_size

16260